<a href="https://colab.research.google.com/github/pedroteixeirazup/Keras/blob/master/KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, MaxPooling2D
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

import time

LOG_DIR = f'{int(time.time())}'

(x_train, y_train), (x_test,y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28,28,1)


In [ ]:
def build_model(hp):
  model = keras.models.Sequential()

  model.add(Conv2D(hp.Int('input_units',min_value=32,max_value=256,step=32), (3, 3), input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  for i in range(hp.Int('n_layers', 1,4)):
      model.add(Conv2D(hp.Int(f'conv_{i}_units',min_value=32,max_value=256,step=32), (3, 3), (3, 3)))
      model.add(Activation('relu'))
  # model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

  model.add(Dense(10))
  model.add(Activation("softmax"))

  model.compile(optimizer="adam",
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])

  return model

tuner = RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 1,
    executions_per_trial = 1,
    directory = LOG_DIR
)

tuner.search(x=x_train,
             y=y_train,
             epochs=1,
             batch_size=64,
             validation_data=(x_test,y_test))

938/938 [==============================] - 124s 132ms/step - loss: 0.6647 - accuracy: 0.8188 - val_loss: 0.4082 - val_accuracy: 0.8569


INFO:tensorflow:Oracle triggered exit


In [ ]:
import pickle

with open(f'tuner_{int(time.time())}.pkl', 'wb') as f:
    pickle.dump(tuner, f)

tuner = pickle.load(open('tuner_1594324820.pkl','rb'))
print(tuner.get_best_hyperparameters()[0].values)
print(tuner.results_summary())
print(tuner.get_best_models()[0].summary())

{'input_units': 192, 'n_layers': 1, 'conv_0_units': 96}


None
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 192)       1920      
_________________________________________________________________
activation (Activation)      (None, 26, 26, 192)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 192)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 96)          165984    
_________________________________________________________________
activation_1 (Activation)    (None, 4, 4, 96)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 10)            